# Quadratic Arithmetic Programs

A quadratic arithmetic program is an arithmetic circuit, specifically a *Rank 1 Constraint System (R1CS)* represented as a set of polynomials. It is derived using *Lagrange interpolation* on a Rank 1 Constraint System. Unlike an R1CS, a Quadratic Arithmetic Program (QAP) can be tested for equality in $\mathcal{O}(1)$ time via the Schwartz-Zippel Lemma. Note that we do have the overhead of "converting" the vectors in polynomials but it is acceptable as we will see toards the end.  
Because a Rank 1 Constraint System is entirely composed of vector operations, we aim to test if
$$
\mathbf{L}\mathbf{a}\circ\mathbf{R}\mathbf{a}\stackrel{?}{=}\mathbf{O}\mathbf{a}
$$
holds in $\mathcal{O}(1)$ time instead of $\mathcal{O}(n)$ time.  
For all math going forward, we assume we are working in a *finite field*, but skip writing $\mod p$ for succinctness.

## Homomorphisms between vector addition and polynomial addition

### Vector addition is homomorphic to polynomial addition
If we take two vectors, interpolate them with polynomials, then add the polynomials together, we get the same polynomial as if we added the vectors together and then interpolated the sum vector.
$$\mathcal{L}(\mathbf{v} + \mathbf{w}) = \mathcal{L}(\mathbf{v}) + \mathcal{L}(\mathbf{w})$$
Also, multiplying a scalar $\lambda$ is essentially adding the vector to itself $\lambda$ times.
$$\mathcal{L}(\lambda \mathbf{v}) = \lambda\mathcal{L}(\mathbf{v})$$


In [ ]:
# Testing the math in Python

import galois
import numpy as np

p = 17
GF = galois.GF(p)

xs = GF(np.array([1,2,3]))

# two arbitrary vectors
v1 =  GF(np.array([4,8,2])) 
v2 =  GF(np.array([1,6,12]))

def L(v):
    return galois.lagrange_poly(xs, v)

assert L(v1 + v2) == L(v1) + L(v2)

So, **The group of vectors under addition in a finite field is homomorphic to the group of polynomials under addition in a finite field.**  
This is critical because **vector equality testing takes $\mathcal{O}(n)$ time, but polynomial equality testing takes $\mathcal{O}(1)$ time.**  

This is what a *Quadratic Arithmetic Program* is.

## A Rank 1 Constraint System in Polynomials
Consider that matrix multiplication between a rectangular matrix and a vector can be written in terms of vector addition and scalar multiplication.
$$
\mathbf{A} \cdot \mathbf{v} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}
\end{bmatrix}
\begin{bmatrix}
v_1\\
v_2\\
v_3\\
v_4
\end{bmatrix}
$$
$$
\mathbf{A}\cdot \mathbf{v} =
\begin{bmatrix}
a_{11}\cdot v_1 + a_{12}\cdot v_2 + a_{13}\cdot v_3 + a_{14}\cdot v_4\\
a_{21}\cdot v_1 + a_{22}\cdot v_2 + a_{23}\cdot v_3 + a_{24}\cdot v_4\\
a_{31}\cdot v_1 + a_{32}\cdot v_2 + a_{33}\cdot v_3 + a_{34}\cdot v_4
\end{bmatrix}
$$
However, we could instead think of splitting matrix $A$ into bunch of vectors as:
$$
\mathbf{A} = \begin{bmatrix}
a_{11} \\
a_{21} \\
a_{31}
\end{bmatrix}
,
\begin{bmatrix}
a_{12} \\
a_{22} \\
a_{32}
\end{bmatrix}
,
\begin{bmatrix}
a_{13} \\
a_{23} \\
a_{33}
\end{bmatrix}
,
\begin{bmatrix}
a_{14} \\
a_{24} \\
a_{34}
\end{bmatrix}
$$
and multiplying each vector by a scalar from the vector $\mathbf{v}$:
$$
\mathbf{A}\cdot \mathbf{v} = \begin{bmatrix}
a_{11} \\
a_{21} \\
a_{31}
\end{bmatrix}\cdot v_1
+
\begin{bmatrix}
a_{12} \\
a_{22} \\
a_{32}
\end{bmatrix}\cdot v_2
+
\begin{bmatrix}
a_{13} \\
a_{23} \\
a_{33}
\end{bmatrix}\cdot v_3
+
\begin{bmatrix}
a_{14} \\
a_{24} \\
a_{34}
\end{bmatrix}\cdot v_4
$$
We will be using this 